# User

In [1]:
class User:
    def __init__(self, phone, password):
        self.__phone = phone
        self.__password = password
        self.__online = False
        self.__connection = None
    
    @property
    def phone(self):
        return self.__phone
    
    @phone.setter
    def phone(self, phone):
        self.__phone = phone
        
    @property
    def online(self):
        return self.__online
    
    @online.setter
    def online(self, online):
        self.__online = online
        
    @property
    def connection(self):
        return self.__connection
    
    @connection.setter
    def connection(self, connection):
        self.__connection = connection
        
    @property
    def pwd(self):
        return self.__password
    
    def validate(self, password):
        return self.__password == password
    

# Server

In [2]:
from socket import socket, AF_INET, SOCK_STREAM
from threading import Thread, Lock

class Server:
    __RSP_OK = '0|ok'
    __RSP_INVALID_CMD = '1|invalid command'
    __RSP_USER_EXIST = '1|user already exists'
    __RSP_INVALID_LOGIN = '1|incorrect username or password'
    __RSP_LGIN_ELSEWHERE = '1|user logged in elsewhere'
    __RSP_NO_SOURCE = '1|no source user'
    __RSP_WRONG_PWD = '2|wrong password'
    __RSP_NO_TARGET = '3|no target user'
    __RSP_OFFLINE = '4|user not online'
    __RSP_NOT_IMPLEMENTED = '5|not implemented'
    
    def __init__(self, port, backlog):
        self.__port = port
        self.__backlog = backlog # Incoming connection queue size
        
        self.__connectionCounter = 1 # Total number of connections handled
        
        self.__incomingsocketserver = socket(AF_INET, SOCK_STREAM)
        self.__outgoingsocketserver = socket(AF_INET, SOCK_STREAM)
        self.__users = {} # dictionary of users
        
    @property
    def users(self):
        return self.__users
    
#-----------------------------------------------------------------------------------    
    #
    # Starts and runs the server
    #
    def runServer(self):
        try:
            self.__incomingsocketserver.bind(('localhost', self.__port))
            self.__outgoingsocketserver.bind(('localhost', self.__port+1))
            self.__incomingsocketserver.listen(self.__backlog)
            self.__outgoingsocketserver.listen(self.__backlog)
            print('Listening on port {}'.format(self.__port))
            
            try:
                self.__displayMessage("Waiting for a Connections...")
                t_in = Thread(target=self.__waitForConnection, args=(self.__incomingsocketserver, 'in',))
                t_in.start()
                t_out = Thread(target=self.__waitForConnection, args=(self.__outgoingsocketserver, 'out',))
                t_out.start()
                
                t_in.join()
                t_out.join()
                
            except Exception as err:
                print(err)
         
        except Exception as err:
            print(err)
            
        finally:
            self.__incomingsocketserver.close()
            self.__outgoingsocketserver.close()
        
        
#-----------------------------------------------------------------------------------
    #
    # Waits for connection and returns it.
    #
    def __waitForConnection(self, server, direction):
        while True:
            client, addr = server.accept()
            self.__displayMessage("Connection {0.connectionCounter} received from: {1}".format(self, str(addr)))
            Thread(target=self.__processConnection, args=(client, direction,)).start()
    
    
#-----------------------------------------------------------------------------------
    def __processConnection(self, connection, direction):
        msg = ""
        user = None
        try:
            msg = connection.recv(1024).decode('UTF-8').strip()
            while msg != '':
                if direction == 'in':
                    response, user = self.__processIncomingPipeMessage(msg)
                else:
                    response, user = self.__processOutgoingPipeMessage(msg, connection)
                self.__sendMessage(response, connection)
                msg = connection.recv(1024).decode('UTF-8').strip()
            
        except Exception as err:
            print(err)


#-----------------------------------------------------------------------------------
    def __closeConnection(self, connection):
        self.__displayMessage("Terminating connection...");
        try:
            connection.close()
        except Exception as err:
            print(err)
            
            
#-----------------------------------------------------------------------------------
    def __sendMessage(self, msg, connection):
        connection.send((msg+'\n').encode('UTF-8'))
        self.__displayMessage("Server>> " + msg)
        
            
#-----------------------------------------------------------------------------------
    def __displayMessage(self, msg):
        print(msg)
        
    
#-----------------------------------------------------------------------------------
    #
    # Process client messages.
    #
    def __processIncomingPipeMessage(self, msg):
        arguments = msg.strip().split('|')
        args = len(arguments)
        
        print("Client>> " + msg)
        
        # Add user
        if arguments[0] is 'A':
            if args >= 3:
                if self.__users.get(arguments[1]) is None:
                    self.__users[arguments[1]] = User(arguments[1], arguments[2])
                    self.__login(arguments[1], arguments[2])
                    return self.__RSP_OK, self.__users[arguments[1]]
                else:
                    return self.__RSP_USER_EXIST, None
            else:
                return self.__RSP_INVALID_CMD, None
        
        # Login
        elif arguments[0] is 'L':
            if args >= 3:
                resp = self.__login(arguments[1], arguments[2])
                if resp == 'err':
                    return self.__RSP_LGIN_ELSEWHERE, None
                elif resp:
                    return self.__RSP_OK, self.__users[arguments[1]]
                else:
                    return self.__RSP_INVALID_LOGIN, None
            else:
                return self.__RSP_INVALID_CMD, None
        
        # Message
        elif arguments[0] is 'M':
            if args >= 5:
                source = self.__users.get(arguments[1], None)
                if source is not None:
                    if source.validate(arguments[2]):
                        target = self.__users.get(arguments[3], None)
                        if target is not None:
                            if target.online:
                                Thread(target=self.__relayMessage, args=(msg, target.connection,)).start()
                                return self.__RSP_OK, source
                            else:
                                return self.__RSP_OFFLINE, None
                        else:
                            return self.__RSP_NO_TARGET, None
                    else:
                        return self.__RSP_WRONG_PWD, None
                else:
                    return self.__RSP_NO_SOURCE, None
            else:
                return self.__RSP_INVALID_CMD, None
            
        # Disconnecting
        elif arguments[0] == 'QUIT':
            user = self.__users[arguments[1]]
            user.online = False
            user.connection.send('QUIT'.encode())
            user.connection = None
            print(user.phone+' disconnected')
            return self.__RSP_OK, None

        # Default
        else:
            return self.__RSP_NOT_IMPLEMENTED, None
        
    
#-----------------------------------------------------------------------------------
    #
    # Process client messages.
    #
    def __processOutgoingPipeMessage(self, msg, connection):
        arguments = msg.strip().split('|')
        args = len(arguments)
        
        print("Client>> " + msg)
        
        # Connect outgoing pipe
        if arguments[0] is 'C':
            if args >= 2:
                u = self.__users.get(arguments[1], None)
                if u is not None:
                    if u.online:
                        u.connection = connection
                        return self.__RSP_OK, u
                    else:
                        return self.__RSP_OFFLINE, None
                else:
                    return self.__RSP_NO_SOURCE, None
            else:
                return self.__RSP_INVALID_CMD, None

        # Default
        else:
            return self.__RSP_NOT_IMPLEMENTED, None
        
        
#-----------------------------------------------------------------------------------
    def __login(self, phone, password):
        u = self.__users.get(phone, None)
        if u is not None:
            if u.validate(password):
                if not u.online:
                    u.online = True
                    print(phone + ' logged in ' + str(u.online))
                    return True
                else:
                    print('User already logged in')
                    return 'err'
            else:
                return False
        else:
            return False
            
            
#-----------------------------------------------------------------------------------
    def __relayMessage(self, msg, connection):
        connection.send(msg.encode())
    
    
    @property
    def connectionCounter(self):
        return self.__connectionCounter
    

In [3]:
server = Server(5000, 30)
server.runServer()

Listening on port 5000
Waiting for a Connections...
Connection 1 received from: ('127.0.0.1', 60798)
Client>> L|303|pwd
Server>> 1|incorrect username or password
Connection 1 received from: ('127.0.0.1', 60802)
Client>> A|303 pwd|pwd
303 pwd logged in True
Server>> 0|ok
Connection 1 received from: ('127.0.0.1', 60804)
Client>> C|303 pwd|pwd
Server>> 0|ok
Connection 1 received from: ('127.0.0.1', 60898)
Client>> A|313|pwd
313 logged in True
Server>> 0|ok
Connection 1 received from: ('127.0.0.1', 60899)
Client>> C|313|pwd
Server>> 0|ok
Client>> M|313|pwd|303 pwd|Hello
Server>> 0|ok
Client>> M|303 pwd|pwd|313|hello back
Server>> 0|ok
Client>> QUIT|313
313 disconnected
Server>> 0|ok
Client>> M|303 pwd|pwd|313|still there?
Server>> 4|user not online
Client>> QUIT|303 pwd
303 pwd disconnected
Server>> 0|ok
Connection 1 received from: ('127.0.0.1', 60931)
Connection 1 received from: ('127.0.0.1', 60932)


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/scott/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/scott/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-607f7af19fd7>", line 69, in __waitForConnection
    client, addr = server.accept()
  File "/Users/scott/opt/anaconda3/lib/python3.7/socket.py", line 212, in accept
    fd, addr = self._accept()
ConnectionAbortedError: [Errno 53] Software caused connection abort
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/scott/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/scott/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-607f7af19fd7>", line 69, in __waitForConnection
    client, addr = server.accept()
  F

KeyboardInterrupt: 